In [20]:
include("MCTS-RF_env.jl")
include("MCTS-RF_agt.jl")

#1局1局の情報をストックする
mutable struct ReplayBuffer
    buffer::Vector{Agent}
    buffer_size::Int
    batch_size::Int
    #count::Int
end

function init_buffer(buffer_size::Int, batch_size::Int)
    return ReplayBuffer([], buffer_size, batch_size)
end


function save_game!(buffer::ReplayBuffer, agt::Agent)
    if length(buffer.buffer) > buffer.buffer_size
        popfirst!(buffer.buffer)
    end
    push!(buffer.buffer, agt)
end
#=
@everywhere function save_game!(buffer, agt::Agent, buffer_size::Int)
    if length(buffer) > buffer_size
        popfirst!(buffer)
    end
    push!(buffer, agt)
end=#

#=function sample_batch!(env::Env, buffer::ReplayBuffer, imag::SharedArray, target::SharedArray)
    games = sample(buffer.buffer, weights([length(agt.history) for agt in buffer.buffer]), buffer.batch_size, replace=false)
    g_turn = [(g, rand(1:length(g.history))) for g in games]
    #imag = SharedArray(zeros(Int, env.input_dim, buffer.batch_size))
    #target = SharedArray(zeros(Float32, env.output, buffer.batch_size))
    @sync @distributed for it in 1:buffer.batch_size
        g, turn = g_turn[it]
        imag[:,it] = make_image(env, g, turn)
        target[:,it] = make_target(env, g, turn)
    end
    #return imag, target
    #return [(make_image(env, g, turn), make_target(env, g, turn)) for (g, turn) in g_turn]
    #return [make_image(env, g, turn) for (g, turn) in g_turn], [make_target(env, g, turn) for (g, turn) in g_turn]
end=#

mutable struct Storage
    storage::Dict{Int, Chain}
    random_out::Chain
    scores::Dict{Vector{Int}, Float32}
end

function init_storage(env)
    #return Storage(Dict(), Chain(Dense(zeros(Float32, env.output,env.input_dim))))
    return Storage(Dict(), Chain(Dense(zeros(Float32, env.output,env.input_dim))), Dict())
end

function latest_model(storage::Storage)
    if(isempty(storage.storage))
        return storage.random_out
    else
        return storage.storage[maximum(keys(storage.storage))]
    end
end


#cpu並列化予定
#=function sample_batch!(env::Env, buffer::ReplayBuffer, scores::Dict{Vector{Int}, Float32})
    games = sample(buffer.buffer, weights([length(agt.history) for agt in buffer.buffer]), buffer.batch_size, replace=false)
    g_turn = [(g, rand(1:length(g.history))) for g in games]
    imag = SharedArray(zeros(Int, env.input_dim, buffer.batch_size))
    target = SharedArray(zeros(Float32, env.output, buffer.batch_size))
    @sync @distributed for it in 1:buffer.batch_size
        g, turn = g_turn[it]
        imag[:,it] = make_image(env, g, turn)
        target[:,it] = make_target(env, g, scores, turn)
    end
    tar_data = sdata(target)
    for it in 1:buffer.batch_size
        g, turn = g_turn[it]
        scores[g.history] = tar_data[end,it]
    end

    return sdata(imag), tar_data
end=#

function sample_batch(env::Env, buffer::ReplayBuffer)
    games = sample(buffer.buffer, weights([length(agt.history) for agt in buffer.buffer]), buffer.batch_size, replace=false)
    g_turn = [(g, rand(1:length(g.history))) for g in games]
    #imag = SharedArray(zeros(Int, env.input_dim, buffer.batch_size))
    #target = SharedArray(zeros(Float32, env.output, buffer.batch_size))
    imag = zeros(Int, env.input_dim, buffer.batch_size)
    target = zeros(Float32, env.output, buffer.batch_size)
    for it in 1:buffer.batch_size
        g, turn = g_turn[it]
        imag[:,it] = make_image(env, g, turn)
        target[:,it] = make_target(env, g, turn)
        println("------------")
        println("sample $(it)")
        println("history: $(g.history),  turn:$(turn)")
        println("target; $(target[:,it])")
    end
    return imag, target
    #return [(make_image(env, g, turn), make_target(env, g, turn)) for (g, turn) in g_turn]
    #return [make_image(env, g, turn) for (g, turn) in g_turn], [make_target(env, g, turn) for (g, turn) in g_turn]
end



#cpu並列化予定
function run_selfplay(env::Env, buffer::ReplayBuffer, storage::Storage)
    for it in 1:env.num_player
        model = latest_model(storage)
        game = play_physics!(env, model)
        println("game$(it): $(game.history)")
        println("child_visit: $(game.child_visit_pi)")
        save_game!(buffer, game)
    end
end

#cpu並列化予定
function run_selfplay(env::Env, buffer::ReplayBuffer, storage::Storage, ratio::Float32)
    for it in 1:env.num_player
        model = latest_model(storage)
        game = play_physics!(env, model, ratio)
        println("game$(it): $(game.history)")
        println("child_visit: $(game.child_visit_pi)")
        save_game!(buffer, game)
    end
end

#gpu並列化予定
#=function loss(image::SharedMatrix{Int}, target::SharedMatrix{Float32}, env::Env, model)
    y1 = model(image)
    return sum([((y1[end,i]-target[end,i])^2 - target[1:end-1,i]' * log.(softmax(y1[1:end-1,i]).+1f-6)) for i in 1:env.batch_size])/env.batch_size + env.C * sum(Flux.params(model)[1].^2)
end=#

sqnorm(x) = sum(abs2, x)

function loss(image::Matrix{Int}, target::Matrix{Float32}, env::Env, model)
    y1 = model(image)
    return sum([((y1[end,i]-target[end,i])^2 - target[1:end-1,i]' * log.(softmax(y1[1:end-1,i]).+1f-6)) for i in 1:env.batch_size])/env.batch_size + env.C * sum(sqnorm, Flux.params(model))
end


tanh10(x) = Float32(10)*tanh(x/10)
tanh2(x) = Float32(2)*tanh(x)

#gpu並列化予定
function train_model!(env::Env, buffer::ReplayBuffer, storage::Storage)
    model = Chain(Dense(env.input_dim, env.middle_dim), Tuple(Chain(Parallel(+, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, env.middle_dim, relu)),Dense(env.middle_dim, env.middle_dim, relu)), identity) for i in 1:env.depth)..., Flux.flatten, Flux.Parallel(vcat, Dense(env.middle_dim, env.act_ind, tanh), Dense(env.middle_dim, 1, tanh10)))

    opt = ADAM()
    #ParameterSchedulers.Scheduler(env.scheduler, Momentum())
    iv_batch = []
    tv_batch = []
    for b_num in 1:env.batch_num
        #image_batch, target_batch = sample_batch!(env, buffer, storage.scores)
        image_batch, target_batch = sample_batch(env, buffer)
        push!(iv_batch, image_batch)
        push!(tv_batch, target_batch)
    end
    l = 0.0
    for it in 1:env.training_step
        for b_num in 1:env.batch_num
            #Flux.train!(loss, Flux.params(model), [(iv_batch[b_num], tv_batch[b_num], env, model)], opt)
            val, grads = Flux.withgradient(Flux.params(model)) do
                loss(iv_batch[b_num],tv_batch[b_num],env,model)
            end
            Flux.Optimise.update!(opt, Flux.params(model), grads)
            l+=val/(env.batch_num*env.training_step)
        end
    end
    storage.storage[env.training_step] = model

    return l
end

train_model! (generic function with 1 method)

In [22]:
env = init_Env(["10", "10", "128", "16", "800", "8", "1", "500", "0.3", "0.25", "50", "4", "20.0", "0.4", "1.0", "0.7", "0.5", "128", "1.25", "0.00001", "0"])
storage = init_storage(env)

Storage(Dict{Int64, Chain}(), Chain(Dense(60 => 7)), Dict{Vector{Int64}, Float32}())

In [23]:
ld = []

for it in 1:10
    println("=============")
    println("it=$(it);")

    replay_buffer = init_buffer(1000, env.batch_size)
    
    if(it<10)
        ratio = Float32(10.0)
        #@time run_selfplay(env, replay_buffer, storage)
        run_selfplay(env, replay_buffer, storage, ratio)
        ll = train_model!(env, replay_buffer, storage)
    else
        #run_selfplay(env, replay_buffer, storage)
        ratio = Float32(10.0)
        if(it<10)
            ratio = Float32(0.2)
        elseif(it<20)
            ratio = Float32(0.05)
        elseif(it<30)
            ratio = Float32(0.01)
        else
            ratio = Float32(0.002)
        end
        run_selfplay(env, replay_buffer, storage, ratio)
        ll = train_model!(env, replay_buffer, storage)
    end
    #@report_call run_selfplay(env, replay_buffer, storage)
    #ll = @report_call train_model!(env, replay_buffer, storage)
    println("loss_average: $(ll)")
    push!(ld,ll)
    if(it%10==0)
        for tes in 1:5
            game = play_physics!(env, latest_model(storage), 1f-6)
            score = calc_score(game.history, env)
            println("$(game.history), score:$(score)")
        end
    end
end

it=1;


game1: [1]
child_visit: Vector{Float32}[[0.228, 0.124, 0.126, 0.206, 0.174, 0.142]]
game2: [2]
child_visit: Vector{Float32}[[0.124, 0.222, 0.204, 0.192, 0.132, 0.126]]
game3: [5, 6, 3, 2, 3, 1, 6, 1, 2]
child_visit: Vector{Float32}[[0.162, 0.126, 0.144, 0.132, 0.31, 0.126], [0.162, 0.16, 0.212, 0.166, 0.0, 0.3], [0.0, 0.252, 0.312, 0.202, 0.234, 0.0], [0.15, 0.252, 0.0, 0.2, 0.246, 0.152], [0.124, 0.144, 0.294, 0.128, 0.126, 0.184], [0.306, 0.176, 0.0, 0.158, 0.152, 0.208], [0.126, 0.124, 0.144, 0.156, 0.158, 0.292], [0.626, 0.374, 0.0, 0.0, 0.0, 0.0], [0.376, 0.624, 0.0, 0.0, 0.0, 0.0]]
game4: [4, 3, 6, 4, 1, 1, 6, 1, 1]
child_visit: Vector{Float32}[[0.124, 0.126, 0.126, 0.376, 0.124, 0.124], [0.222, 0.196, 0.238, 0.0, 0.15, 0.194], [0.23, 0.152, 0.0, 0.15, 0.166, 0.302], [0.0, 0.214, 0.274, 0.324, 0.188, 0.0], [0.29, 0.18, 0.15, 0.0, 0.23, 0.15], [0.278, 0.14, 0.202, 0.124, 0.128, 0.128], [0.126, 0.128, 0.164, 0.15, 0.174, 0.258], [0.576, 0.424, 0.0, 0.0, 0.0, 0.0], [0.624, 0.376, 0.

game6: [5, 6, 3, 2, 6, 3, 1, 2, 2]
child_visit: Vector{Float32}[[0.134, 0.124, 0.162, 0.18, 0.258, 0.142], [0.186, 0.158, 0.15, 0.19, 0.0, 0.316], [0.0, 0.188, 0.326, 0.276, 0.21, 0.0], [0.23, 0.288, 0.0, 0.182, 0.15, 0.15], [0.16, 0.168, 0.126, 0.15, 0.132, 0.264], [0.0, 0.252, 0.364, 0.196, 0.188, 0.0], [0.614, 0.386, 0.0, 0.0, 0.0, 0.0], [0.414, 0.586, 0.0, 0.0, 0.0, 0.0], [0.376, 0.624, 0.0, 0.0, 0.0, 0.0]]
game7: [1]
child_visit: Vector{Float32}[[0.282, 0.124, 0.206, 0.124, 0.134, 0.13]]
game8: [1]
child_visit: Vector{Float32}[[0.286, 0.132, 0.158, 0.124, 0.174, 0.126]]
game9: [4, 2, 5, 3, 6, 5, 1, 1, 1, 2]
child_visit: Vector{Float32}[[0.128, 0.126, 0.126, 0.268, 0.156, 0.196], [0.15, 0.322, 0.208, 0.0, 0.156, 0.164], [0.13, 0.128, 0.124, 0.124, 0.368, 0.126], [0.15, 0.184, 0.282, 0.156, 0.0, 0.228], [0.178, 0.172, 0.0, 0.162, 0.154, 0.334], [0.0, 0.188, 0.264, 0.258, 0.29, 0.0], [0.542, 0.458, 0.0, 0.0, 0.0, 0.0], [0.604, 0.396, 0.0, 0.0, 0.0, 0.0], [0.616, 0.384, 0.0, 0.0, 0.0,

------------
sample 1
history: [4, 3, 6, 4, 1, 1, 6, 1, 1],  turn:7
target; Float32[0.126, 0.128, 0.164, 0.15, 0.174, 0.258, 4.636553]


------------
sample 2
history: [4, 2, 5, 3, 6, 5, 1, 1, 1, 2],  turn:1
target; Float32[0.128, 0.126, 0.126, 0.268, 0.156, 0.196, -0.89178914]
------------
sample 3
history: [1],  turn:1
target; Float32[0.286, 0.132, 0.158, 0.124, 0.174, 0.126, 4.636553]


...

[2], score:-1.389799


[6, 3, 2, 1], score:-5.1887093


[2], score:-1.389799


[1], score:4.636553


[3, 6, 4, 5, 4, 2, 1, 1, 2, 1], score:-4.341542
